# 🧪 Local Git Pre-Commit Hook Demo
This notebook demonstrates how to automatically run tests and format code using a Git `pre-commit` hook. The hook runs `pytest` and `black` before each commit to ensure code quality.

In [2]:
import os
import subprocess
import tempfile
from pathlib import Path

def run(cmd, cwd=None, label=None):
    """Run a shell command, print a heading, and filter Git hints."""
    if label:
        print(f"\n🔹 {label}")
    print(f"$ {cmd}")
    try:
        result = subprocess.run(cmd, shell=True, cwd=cwd, check=True,
                                stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        lines = [line for line in result.stdout.splitlines() if not line.strip().startswith("hint:")]
        print("\n".join(lines))
        return result.stdout
    except subprocess.CalledProcessError as e:
        lines = [line for line in e.stdout.splitlines() if not line.strip().startswith("hint:")]
        print("\n".join(lines))
        raise

# Create a temporary Git project
with tempfile.TemporaryDirectory() as tempdir:
    project = Path(tempdir) / "myproject"
    os.makedirs(project)
    print(f"\n📁 Created temporary Git repository at: {project}\n")

    # Step 1: Initialize Git and set identity
    run("git init", cwd=project, label="Initializing Git Repository")
    run('git config user.name "CI Tester"', cwd=project, label="Setting Git User Name")
    run('git config user.email "ci@test.local"', cwd=project, label="Setting Git User Email")

    # Step 2: Create intentionally misformatted but valid Python test code
    test_code = """\
def add( a ,b ):
  return a+b

def test_answer( ):
  print("Testing add(1, 3) == 4")
  assert add(1,3)==4
"""
    test_file = project / "test_example.py"
    test_file.write_text(test_code)
    print("\n" + "-"*80)
    print("📄 test_example.py before formatting:")
    print("-"*80)
    print(test_code)

    # Step 3: Run black manually and compare formatting
    run("black --quiet test_example.py", cwd=project, label="Running black (manual format before commit)")
    formatted = test_file.read_text()
    print("\n" + "-"*80)
    print("📄 test_example.py after formatting:")
    print("-"*80)
    print(formatted)

    # Step 4: Create the Git pre-commit hook
    hook_code = """\
#!/bin/sh
pytest || exit 1
black .
"""
    hook_path = project / ".git/hooks/pre-commit"
    hook_path.write_text(hook_code)
    hook_path.chmod(0o755)
    print("\n🔧 Created pre-commit hook:\n")
    print(hook_code)

    # Step 5: Manual test run (for visible output)
    run("pytest", cwd=project, label="Manual Test Run")

    # Step 6: Add and commit — hook will be triggered
    run("git add .", cwd=project, label="Staging Files")
    try:
        run("git commit -m 'test pre-commit hook'", cwd=project, label="Committing with Pre-Commit Hook")
        print("\n✅ Commit succeeded. Git hook ran correctly.\n")
    except subprocess.CalledProcessError:
        print("\n❌ Commit failed. The pre-commit hook blocked the commit.\n")



📁 Created temporary Git repository at: /var/folders/4d/v62x5gg540g5q0n2_dmlyd60000c0j/T/tmpmxcaz6qs/myproject


🔹 Initializing Git Repository
$ git init
Leeres Git-Repository in /private/var/folders/4d/v62x5gg540g5q0n2_dmlyd60000c0j/T/tmpmxcaz6qs/myproject/.git/ initialisiert

🔹 Setting Git User Name
$ git config user.name "CI Tester"


🔹 Setting Git User Email
$ git config user.email "ci@test.local"


--------------------------------------------------------------------------------
📄 test_example.py before formatting:
--------------------------------------------------------------------------------
def add( a ,b ):
  return a+b

def test_answer( ):
  print("Testing add(1, 3) == 4")
  assert add(1,3)==4


🔹 Running black (manual format before commit)
$ black --quiet test_example.py


--------------------------------------------------------------------------------
📄 test_example.py after formatting:
--------------------------------------------------------------------------------
def add(